In [222]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [223]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6933, 7)

In [224]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6914, 5)

In [225]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6939, 9)

fin empresas y comienzo de rescate de pacientes


In [226]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1628309, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [227]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1192491, 4)

In [228]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1192491, 12)

In [230]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [231]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [232]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [233]:
#pacientes_empresas22.shape

In [234]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [235]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [236]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [237]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "VidaCel Colaboradores"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [238]:
Pacientes_Archivo.head(10)

,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,apellidos,nacimiento,Edad,email,direccion,comuna,Isapre,celular
0,Vidacel,19686491,1,19686491,1,19686491-1,Masculino,Titular,NaN,VICENTE IGNACIO,ARANCIBIA BACKHOUSE,1997-05-23,25.463014,vicente.arancibia.b@gmail.com,Francisco Bulnes Correa 906,Las Condes,SI,5.698762e+10
1,Vidacel,27028240,7,27028240,7,27028240-7,Masculino,Titular,NaN,PABLO ALEJANDRO,DE SIMONE,1978-08-21,44.230137,pabloadesimone@gmail.com,Camino el Huinganal 3805,Lo Barnechea,SI,9.367339e+08
2,Vidacel,7396233,1,7396233,1,7396233-1,Femenino,Titular,NaN,MARIANNE,ROBERTS VARGAS,1971-09-28,51.131507,mariannerobertsvargas@gmail.com,Camino Mirasol 1417,Las Condes,SI,9.840132e+07
3,Vidacel,15064898,K,15064898,K,15064898-K,M,Titular,NaN,CRISTIAN JAVIER,FLORES SILVA,1983-03-14,38.000000,krisfs@gmail.com,Alberdi 1206 Casa 32,Quillota,CRUZ BLANCA,9.656784e+08
4,Vidacel,15064898,k,16819720,9,16819720-9,F,Conyuge,NaN,Camila,Solis,1988-03-03,34.000000,krisfs@gmail.com,Alberdi 1206,Quillota,Fonasa,9.656784e+08
5,Vidacel,15064898,k,21724467,6,21724467-6,F,Hija,NaN,Fernanda,Flores,2006-12-09,17.000000,krisfs@gmail.com,Alberdi 1206,Quillota,Fonasa,9.656784e+08
6,Vidacel,15064898,k,27753598,K,27753598-K,F,Hija,NaN,Dominga,Flores,2022-03-26,0.000000,krisfs@gmail.com,Alberdi 1206,Quillota,Fonasa,9.656784e+08
7,Vidacel,10614179,7,10614179,7,10614179-7,f,Titular,NaN,Jessica,Muñoz Araneda,1967-05-06,55.000000,jessicamunozaraneda@gmail.com,"Santo Domingo 51, Parque del Sol",Maipu,No,9.544475e+08
8,Vidacel,10614179,7,21162018,8,21162018-8,F,HiJa,NaN,Renata,Chacon Muñoz,2002-11-11,19.000000,renata.isidorachacon@gmail.com,Santo domingo 51,Maipu,fonasa,9.501578e+08
9,Vidacel,10614179,7,19112799,4,19112799-4,M,Hijo,NaN,Nicolas,Chacon Muñoz,1995-10-25,26.000000,n.chacon95.nc@gmail.com,Santo domingo 51,Maipu,BANMEDICA,9.885371e+08


In [239]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [240]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,apellidos,nacimiento,Edad,email,direccion,comuna,Isapre,celular,rut
0,Vidacel,19686491,1,19686491,1,19686491-1,Masculino,Titular,NaN,VICENTE IGNACIO,ARANCIBIA BACKHOUSE,1997-05-23,25.463014,vicente.arancibia.b@gmail.com,Francisco Bulnes Correa 906,Las Condes,SI,5.698762e+10,196864911
1,Vidacel,27028240,7,27028240,7,27028240-7,Masculino,Titular,NaN,PABLO ALEJANDRO,DE SIMONE,1978-08-21,44.230137,pabloadesimone@gmail.com,Camino el Huinganal 3805,Lo Barnechea,SI,9.367339e+08,270282407
2,Vidacel,7396233,1,7396233,1,7396233-1,Femenino,Titular,NaN,MARIANNE,ROBERTS VARGAS,1971-09-28,51.131507,mariannerobertsvargas@gmail.com,Camino Mirasol 1417,Las Condes,SI,9.840132e+07,73962331
3,Vidacel,15064898,K,15064898,K,15064898-K,M,Titular,NaN,CRISTIAN JAVIER,FLORES SILVA,1983-03-14,38.000000,krisfs@gmail.com,Alberdi 1206 Casa 32,Quillota,CRUZ BLANCA,9.656784e+08,15064898K
4,Vidacel,15064898,k,16819720,9,16819720-9,F,Conyuge,NaN,Camila,Solis,1988-03-03,34.000000,krisfs@gmail.com,Alberdi 1206,Quillota,Fonasa,9.656784e+08,168197209


In [241]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16532\485331713.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,apellidos,nacimiento,Edad,email,direccion,comuna,Isapre,celular,rut
0,Vidacel,19686491,1,19686491,1,19686491-1,Masculino,Titular,NaN,VICENTE IGNACIO,ARANCIBIA BACKHOUSE,1997-05-23,25.463014,vicente.arancibia.b@gmail.com,Francisco Bulnes Correa 906,Las Condes,SI,5.698762e+10,196864911
1,Vidacel,27028240,7,27028240,7,27028240-7,Masculino,Titular,NaN,PABLO ALEJANDRO,DE SIMONE,1978-08-21,44.230137,pabloadesimone@gmail.com,Camino el Huinganal 3805,Lo Barnechea,SI,9.367339e+08,270282407
2,Vidacel,7396233,1,7396233,1,7396233-1,Femenino,Titular,NaN,MARIANNE,ROBERTS VARGAS,1971-09-28,51.131507,mariannerobertsvargas@gmail.com,Camino Mirasol 1417,Las Condes,SI,9.840132e+07,73962331
3,Vidacel,15064898,K,15064898,K,15064898-K,M,Titular,NaN,CRISTIAN JAVIER,FLORES SILVA,1983-03-14,38.000000,krisfs@gmail.com,Alberdi 1206 Casa 32,Quillota,CRUZ BLANCA,9.656784e+08,15064898K
4,Vidacel,15064898,k,16819720,9,16819720-9,F,Conyuge,NaN,Camila,Solis,1988-03-03,34.000000,krisfs@gmail.com,Alberdi 1206,Quillota,Fonasa,9.656784e+08,168197209


In [242]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16532\2118328728.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16532\2118328728.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [243]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(16, 29)

In [244]:
Pacientes_con_id.head()

,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,Vidacel,19686491,1,19686491,1,19686491-1,Masculino,Titular,NaN,Vicente Ignacio,...,True,0018c00002bI8XgAAK,196864911,Vicente Ignacio Arancibia Backhouse,RUT,None,None,None,Mujer,None
1,Vidacel,27028240,7,27028240,7,27028240-7,Masculino,Titular,NaN,Pablo Alejandro,...,True,0018c00002eiPLDAA2,270282407,Pablo Alejandro De Simone,RUT,None,None,None,Hombre,None
2,Vidacel,7396233,1,7396233,1,7396233-1,Femenino,Titular,NaN,Marianne,...,True,0018c00002eiPLEAA2,73962331,Marianne Roberts Vargas,RUT,None,None,None,Mujer,None
3,Vidacel,15064898,K,15064898,K,15064898-K,M,Titular,NaN,Cristian Javier,...,True,0018c00002eUAcwAAG,15064898K,Cristian Javier Flores Silva,RUT,None,None,None,Hombre,None
4,Vidacel,15064898,k,16819720,9,16819720-9,F,Conyuge,NaN,Camila,...,True,0018c00002eiPLFAA2,168197209,Camila Solis,RUT,None,None,None,Mujer,None


In [246]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"].astype(str).str.upper()
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"].apply(str).str.strip().replace("+","")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtsEAE'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,BillingState,Phone,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
11,False,Yoselin,Caamaño Valdes,153749671,joselincaamano@gmail.com,1983-01-24,Mujer,Dieho Duble Urrutia N°4202,Maipu,942183518.0,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
13,False,Ana Maria,Sabbatino Mancilla,79320331,asabbatino@gmail.com,1959-03-14,Mujer,Jose Zapiola N°8021 E,La Reina,56963353222.0,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
14,False,Paula,Vera Cruz,16459789K,veracruzpaula@gmail.com,1987-04-22,Mujer,"Antonio Varas N° 60, Dpto. 413",Providencia,976958579.0,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
15,False,Norma,Wigolorchew Martin,50268063,normawigo@gmail.com,1947-04-25,Mujer,"Robles N° 13169, Dpto. 31",Lo Barnechea,56992226731.0,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [247]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "45",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [248]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
160458,a1Y8c00000C6VtrEAF,0018c00002amQ2QAAU,a1W8c0000083ilIEAQ,VidaCel Colaboradores,0018c00002cXJfJAAW,Colaboradores VidaCel S.A.,Sin Holding Asociado,VidaCel Colaboradores,Colaboradores VidaCel S.A.,Sin Póliza Asociada,EMP6995,A-1626
160459,a1Y8c00000C6VtsEAF,0018c00002amQonAAE,a1W8c0000083ilIEAQ,VidaCel Colaboradores,0018c00002cXJfJAAW,Colaboradores VidaCel S.A.,Sin Holding Asociado,VidaCel Colaboradores,Colaboradores VidaCel S.A.,Sin Póliza Asociada,EMP6995,A-1626
160460,a1Y8c00000C6VtuEAF,0018c00002amRGlAAM,a1W8c0000083ilIEAQ,VidaCel Colaboradores,0018c00002cXJfJAAW,Colaboradores VidaCel S.A.,Sin Holding Asociado,VidaCel Colaboradores,Colaboradores VidaCel S.A.,Sin Póliza Asociada,EMP6995,A-1626
1177866,a1Y8c00000DzQeiEAF,0018c00002egmVNAAY,a1W8c0000083ilIEAQ,VidaCel Colaboradores,0018c00002cXJfJAAW,Colaboradores VidaCel S.A.,Sin Holding Asociado,VidaCel Colaboradores,Colaboradores VidaCel S.A.,Sin Póliza Asociada,EMP6995,A-1626
1177867,a1Y8c00000DzQejEAF,0018c00002eUDL6AAO,a1W8c0000083ilIEAQ,VidaCel Colaboradores,0018c00002cXJfJAAW,Colaboradores VidaCel S.A.,Sin Holding Asociado,VidaCel Colaboradores,Colaboradores VidaCel S.A.,Sin Póliza Asociada,EMP6995,A-1626


In [249]:
campaña_seleccionada.shape

(139, 12)

In [250]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [251]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [252]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
